In [16]:
#! python2.7 halflife-regression-master/experiment.py -m leitner halflife-regression-master/settles.acl16.learning_traces.13m.csv 

method = "leitner"
reading data...0...1000000...2000000...3000000...4000000...5000000...6000000...7000000...8000000...9000000...10000000...11000000...12000000...done!
|train| = 11568803
|test|  = 1285423
test	458465134.6 (p=216411.3, h=458248723.3, l2=0.0)	mae(p)=0.235	cor(p)=0.056	mae(h)=147.926	cor(h)=-0.105


In [17]:
#! python2.7 halflife-regression-master/experiment.py -m pimsleur halflife-regression-master/settles.acl16.learning_traces.13m.csv 

method = "pimsleur"
reading data...0...1000000...2000000...3000000...4000000...5000000...6000000...7000000...8000000...9000000...10000000...11000000...12000000...done!
|train| = 11568803
|test|  = 1285423
test	502841296.0 (p=506342.9, h=502334953.1, l2=0.0)	mae(p)=0.445	cor(p)=0.022	mae(h)=155.459	cor(h)=-0.112


In [18]:
#! python2.7 halflife-regression-master/experiment.py -m hlr halflife-regression-master/settles.acl16.learning_traces.13m.csv 

method = "hlr"
reading data...0...1000000...2000000...3000000...4000000...5000000...6000000...7000000...8000000...9000000...10000000...11000000...12000000...done!
|train| = 11568803
|test|  = 1285423
test	192022901.0 (p=109047.8, h=191913750.3, l2=102.9)	mae(p)=0.129	cor(p)=0.038	mae(h)=117.551	cor(h)=0.202


In [31]:
import dask.dataframe as dd

In [32]:
df_hlr = dd.read_csv('results/hlr.settles.acl16.learning_traces.13m.preds', sep='\t')
df_leitner = dd.read_csv('results/leitner.settles.acl16.learning_traces.13m.preds', sep='\t')
df_pimsleur = dd.read_csv('results/pimsleur.settles.acl16.learning_traces.13m.preds', sep='\t')

In [33]:
df_hlr.columns = ['p', 'pp_hlr', 'h_hlr', 'hh_hlr', 't', 'right', 'seen', 'lang', 'user_id', 'timestamp', 'lexeme_tag']
df_leitner.columns = ['p', 'pp_l', 'h_l', 'hh_l', 't', 'right', 'seen', 'lang', 'user_id', 'timestamp', 'lexeme_tag']
df_pimsleur.columns = ['p', 'pp_p', 'h_p', 'hh_p', 't', 'right', 'seen', 'lang', 'user_id', 'timestamp', 'lexeme_tag']

In [34]:
df = dd.concat([df_leitner, df_pimsleur, df_hlr])

In [35]:
df.columns

Index(['p', 'pp_l', 'h_l', 'hh_l', 't', 'right', 'seen', 'lang', 'user_id',
       'timestamp', 'lexeme_tag', 'pp_p', 'h_p', 'hh_p', 'pp_hlr', 'h_hlr',
       'hh_hlr'],
      dtype='object')

In [36]:
df['delta_by_h_p'] = df.t/df.hh_p
df['delta_by_h_l'] = df.t/df.hh_l
df['delta_by_h_hlr'] = df.t/df.hh_hlr

In [37]:
data = df.compute()

In [39]:
len(data.lexeme_tag.unique())

13304

In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot
pd.set_option('display.max_columns', 500)
import missingno as msno
import matplotlib.pyplot as plt
from anytree.resolver import Resolver
from anytree.importer import DictImporter
from anytree import RenderTree
from anytree.search import findall
import re
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from collections import OrderedDict
warnings.filterwarnings('ignore')

In [41]:
def ql_calc(u, y):
    # Ref: McCullagh and Nelder, 1989, p. 325
    return y*np.log(u/(1-u)) + np.log(1-u)


def qic(model, data, params, scale, cov_params):

    varfunc = model.family.variance

    from patsy import dmatrices
    y, X = dmatrices(formula, data)

    y = y.ravel()

    lpr = np.dot(X, params)
    means = model.family.link.inverse(lpr)

    # omega^-1 is the model-based covariance assuming independence    
    dmat = model.mean_deriv(X, lpr)
    omega = np.dot(dmat.T, dmat)

    # Analytical evaluation of Quasi-likelihood for Binomial Distribution
    # Ref: W. Pan (2001). Akaike’s information criterion in generalized estimating equations. Biometrics, p.121
    # Ref: McCullagh and Nelder, 1989, p. 325
    
    ql = np.sum(ql_calc(means, y))/scale
    

    qicu = -2 * ql + 2 * X.shape[1]
    qic = -2 * ql + 2 * np.trace(np.dot(omega, cov_params))

    return ql, qic, qicu

In [42]:
# Student and Concept as random effects
def non_nested(formula, data, hyp):
    data_copy = data.copy()
    data_copy["StudentID_Concept"] = data_copy.user_id + data_copy.lexeme_tag

    model_student = sm.GEE.from_formula(formula, groups="user_id",
                        data=data_copy, family=sm.families.Binomial(), cov_struct=sm.cov_struct.Independence())
    result_student = model_student.fit()

    model_concept = sm.GEE.from_formula(formula, groups="lexeme_tag",
                        data=data_copy, family=sm.families.Binomial(), cov_struct=sm.cov_struct.Independence())
    result_concept = model_concept.fit()

    model_student_concept = sm.GEE.from_formula(formula, groups="StudentID_Concept",
                        data=data_copy, family=sm.families.Binomial(), cov_struct=sm.cov_struct.Independence())
    result_student_concept = model_student_concept.fit()

    final_cov = result_student.normalized_cov_params + result_concept.normalized_cov_params - result_student_concept.normalized_cov_params

    print("ql: {0:.2f}  qic:{1:.2f}  qicu:{2:.2f}".format(*qic(model_concept, data, 
                                                      result_concept.params, result_concept.scale, 
                                                      final_cov)))


    if type(hyp) is list:
        output = []
        for i in hyp:
            output.append(result_student.wald_test(i, cov_p = final_cov).pvalue.item())
        return output
    else:
        return result_student.wald_test(hyp, cov_p = final_cov)

In [43]:
def run_gee(data,formula):
    
    list_vars = formula.split(" ~ ")[1].split(' + ')
    model_concept = sm.GEE.from_formula(formula, groups="lexeme_tag",
    data=data, family=sm.families.Binomial(), cov_struct=sm.cov_struct.Independence())
    result_concept = model_concept.fit()
        
    output = OrderedDict()
    for var in list_vars:
        estimate = result_concept.params[var]
        pvalue = non_nested(formula, data,'{} = 0'.format(var)).pvalue.item()
        
        output["estimate_{}".format(var)] = estimate
        output["pvalue_{}".format(var)] = pvalue
        
    return output

### Sucess ~ OppItem

In [44]:
formula  = "p ~ seen"

In [45]:
%%time
run_gee(data,formula)

ql: -2249754.98  qic:4499686.59  qicu:4499513.97


OrderedDict([('estimate_seen', -0.0003111143509610103),
             ('pvalue_seen', 0.07738696147002817)])

### Pimsleur

In [46]:
formula  = "p ~ delta_by_h_p" # delta and h were rounded to 4 decimals possible loss of information

In [47]:
%%time
run_gee(data,formula)

ql: -749818.55  qic:1499645.02  qicu:1499641.11
CPU times: user 3min 45s, sys: 18.4 s, total: 4min 3s
Wall time: 3min 6s


OrderedDict([('estimate_delta_by_h_p', -2.3841864638480565e-05),
             ('pvalue_delta_by_h_p', 3.465661977656762e-35)])

In [52]:
formula  = "p ~ pp_p"

In [53]:
%%time
run_gee(data,formula)

ql: -749682.52  qic:1499375.31  qicu:1499369.04
CPU times: user 3min 46s, sys: 19.9 s, total: 4min 6s
Wall time: 3min 10s


OrderedDict([('estimate_pp_p', 0.13950174105671806),
             ('pvalue_pp_p', 1.2029193446696543e-11)])

### Leitner

In [48]:
formula  = "p ~ delta_by_h_l" # delta and h were rounded to 4 decimals possible loss of information

In [49]:
%%time
run_gee(data,formula)

ql: -749676.35  qic:1499364.94  qicu:1499356.71
CPU times: user 3min 45s, sys: 18.2 s, total: 4min 3s
Wall time: 3min 6s


OrderedDict([('estimate_delta_by_h_l', -0.0039878943529905895),
             ('pvalue_delta_by_h_l', 7.227240032088166e-12)])

In [54]:
formula  = "p ~ pp_l"

In [55]:
%%time
run_gee(data,formula)

ql: -747982.87  qic:1495974.80  qicu:1495969.74
CPU times: user 3min 47s, sys: 17.9 s, total: 4min 4s
Wall time: 3min 8s


OrderedDict([('estimate_pp_l', 0.44121161309096096),
             ('pvalue_pp_l', 3.356719984743912e-104)])

### Half-Life Regression

In [50]:
formula  = "p ~ delta_by_h_hlr" # delta and h were rounded to 4 decimals possible loss of information

In [51]:
%%time
run_gee(data,formula)

ql: -749514.56  qic:1499037.35  qicu:1499033.11
CPU times: user 3min 42s, sys: 19.3 s, total: 4min 1s
Wall time: 3min 3s


OrderedDict([('estimate_delta_by_h_hlr', -0.323008263766238),
             ('pvalue_delta_by_h_hlr', 1.3848529174968613e-49)])

In [56]:
formula  = "p ~ pp_hlr"

In [57]:
%%time
run_gee(data,formula)

ql: -749313.32  qic:1498634.79  qicu:1498630.63
CPU times: user 3min 46s, sys: 18.8 s, total: 4min 4s
Wall time: 3min 7s


OrderedDict([('estimate_pp_hlr', 0.81708426316908),
             ('pvalue_pp_hlr', 5.069376721969455e-64)])

**Observation:**  There is a clear evidence of using decay due to spacing in predicting student's success. However, there is no strong reason to switch to half-life regression (compute intense method) from Pimsleur or Leitner's method of decaying. Leitner's method has the lowest QIC for this dataset.